In [ ]:
import requests
import os
from PIL import Image
from io import BytesIO

CLIENT_ID = 'u-s4t2ud-cb23abb2c782177b3fdc9c9f0317dafa3e63601a6c0afdcc71693f661a4c5893'
CLIENT_SECRET = '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'

token_url = "https://api.intra.42.fr/oauth/token"
token_data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
}
response = requests.post(token_url, data=token_data)
access_token = response.json()['access_token']

headers = {'Authorization': f'Bearer {access_token}'}
all_users = []
page = 1
while True:
    users_url = f"https://api.intra.42.fr/v2/campus/35/users?page={page}&per_page=100"
    response = requests.get(users_url, headers=headers)
    data = response.json()
    if not data:
        break
    all_users.extend(data)
    print(f"Fetched page {page} with {len(data)} users")
    page += 1

os.makedirs('42_profiles', exist_ok=True)

for user in all_users:
    login = user['login']
    image_url = user['image']['link'] if user['image'] and 'link' in user['image'] else None
    if not image_url:
        print(f"No image URL for {login}, skipping")
        continue
    try:
        img_data = requests.get(image_url, timeout=10).content
        img = Image.open(BytesIO(img_data))
        img.save(f"42_profiles/{login}.jpg")
        print(f"Downloaded: {login}")
    except Exception as e:
        print(f"Failed to download/save image for {login}: {e}")


In [ ]:
import os
import shutil
source_folder = "42_profiles"
target_folder = "data"
os.makedirs(target_folder, exist_ok=True)
for file in os.listdir(source_folder):
    if file.endswith(".jpg"):
        student_name = file.split('.')[0]
        student_folder = os.path.join(target_folder, student_name)
        os.makedirs(student_folder, exist_ok=True)
        shutil.copy(os.path.join(source_folder, file), os.path.join(student_folder, file))
print("Dataset folder structure created :white_check_mark:")

In [ ]:
import cv2
import os
from PIL import Image

source_folder = 'data'

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

for class_folder in os.listdir(source_folder):
    class_folder_path = os.path.join(source_folder, class_folder)
    if not os.path.isdir(class_folder_path):
        continue

    for filename in os.listdir(class_folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            try:
                img_path = os.path.join(class_folder_path, filename)
                img = cv2.imread(img_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
                if len(faces) == 0:
                    print(f":x: No face found in {filename} inside {class_folder}")
                    continue

                x, y, w, h = faces[0]
                face = img[y:y+h, x:x+w]

                face_image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

                new_filename = filename.rsplit('.', 1)[0] + '_cropped.' + filename.rsplit('.', 1)[1]
                save_path = os.path.join(class_folder_path, new_filename)

                face_image.save(save_path)
                print(f":white_check_mark: Cropped and saved: {new_filename} in {class_folder}")

            except Exception as e:
                print(f":warning: Error processing {filename} in {class_folder}: {e}")


In [ ]:
import cv2
import os
from PIL import Image

source_folder = 'data'

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

for class_folder in os.listdir(source_folder):
    class_folder_path = os.path.join(source_folder, class_folder)
    if not os.path.isdir(class_folder_path):
        continue 

    for filename in os.listdir(class_folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            try:
                img_path = os.path.join(class_folder_path, filename)
                img = cv2.imread(img_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
                if len(faces) == 0:
                    print(f":x: No face found in {filename} inside {class_folder}")
                    continue
                x, y, w, h = faces[0]
                face = img[y:y+h, x:x+w]
                face_image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
                new_filename = filename.rsplit('.', 1)[0] + '_cropped.' + filename.rsplit('.', 1)[1]
                save_path = os.path.join(class_folder_path, new_filename)
                face_image.save(save_path)
                print(f":white_check_mark: Cropped and saved: {new_filename} in {class_folder}")
                pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                flipped_img = pil_img.transpose(method=Image.FLIP_LEFT_RIGHT)
                flipped_filename = filename.rsplit('.', 1)[0] + '_flip.' + filename.rsplit('.', 1)[1]
                flipped_img.save(os.path.join(class_folder_path, flipped_filename))
                print(f":white_check_mark: Flipped image saved: {flipped_filename} in {class_folder}")
                rotated_img = pil_img.rotate(-90, expand=True)
                rotated_filename = filename.rsplit('.', 1)[0] + '_rot.' + filename.rsplit('.', 1)[1]
                rotated_img.save(os.path.join(class_folder_path, rotated_filename))
                print(f":white_check_mark: Rotated image saved: {rotated_filename} in {class_folder}")

            except Exception as e:
                print(f":warning: Error processing {filename} in {class_folder}: {e}")


In [ ]:
import pandas as pd
from pathlib import Path

path = Path('data')
files = list(path.glob("*.jpg"))
df = pd.DataFrame({
    'fname': [f.name for f in files],
    'label': [f.stem for f in files]
})

In [ ]:
import shutil
from pathlib import Path

images_dir = Path('42_profiles')

for img_path in images_dir.iterdir():
    if img_path.is_file():
        subdir = images_dir / img_path.stem
        subdir.mkdir(exist_ok=True)
        target_path = subdir / img_path.name
        shutil.move(str(img_path), str(target_path))

print("Done! Each image has been moved into its own folder.")


In [ ]:
!pip install --quiet fastai
!pip install --quiet --upgrade pip

from fastai.vision.all import *
from pathlib import Path
from collections import defaultdict

data_path = Path('data')

all_files = get_image_files(data_path)

class_counts = defaultdict(list)
for f in all_files:
    class_counts[f.parent.name].append(f)
filtered_files = [f for cls, files in class_counts.items() if len(files) > 1 for f in files]

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=lambda _: filtered_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=lambda f: f.parent.name,
     item_tfms=Resize(300)
)

dls = dblock.dataloaders(data_path, bs=32)

dls.show_batch(max_n=9, figsize=(6,6))

In [ ]:
from fastai.vision.utils import verify_image
from fastai.vision.all import *
from pathlib import Path

data_path = Path('data')
all_files = get_image_files(data_path)

for f in all_files:
    try:
        verify_image(f)
    except Exception as e:
        print(f"Removing {f} due to error: {e}")
        f.unlink()


In [ ]:
dls.show_batch(max_n=12, figsize=(10, 10))


In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(12)
learn.export('/kaggle/working/model_last.pkl')
print("✅ Model exported to /kaggle/working/model_last.pkl")

In [ ]:
!pip install facenet-pytorch

from facenet_pytorch import MTCNN
from PIL import Image
import torch

mtcnn = MTCNN(keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')

def crop_face(image_path, save_path=None):
    img = Image.open(image_path)
    boxes, _ = mtcnn.detect(img)
    if boxes is None or len(boxes) == 0:
        print(f"No face found in {image_path}")
        return None
    box = boxes[0]
    left, top, right, bottom = [int(x) for x in box]
    face_img = img.crop((left, top, right, bottom))
    if save_path:
        face_img.save(save_path)
    return face_img
cropped_face = crop_face('/kaggle/input/4654645/IMG_6628.jpg')

if cropped_face:
    pred_class, pred_idx, probs = learn.predict(cropped_face)
    print(f"Predicted student: {pred_class}, Probability: {probs[pred_idx]:.2f}")
else:
    print("No face detected; cannot predict.")
